In [4]:
from numpy import save, load
from pandas import read_csv
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras import Model
from keras.layers import Conv1D, SpatialDropout1D
from keras.layers import Convolution1D, Dense
from keras.models import Input, Model
from keras.layers import LSTM, Conv1D, concatenate,GlobalMaxPooling1D,GlobalAveragePooling1D,TimeDistributed, MaxPooling1D
import tensorflow as tf
import numpy as np
!pip install keras-tcn
from tcn import TCN

In [72]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'  #change dir to your project folder

import numpy as np
x_train = np.load('gdrive/My Drive/dataset/dataset6/trainX.npy')
y_train = np.load('gdrive/My Drive/dataset/dataset6/trainy.npy')
x_test = np.load('gdrive/My Drive/dataset/dataset6/testX.npy')
y_test = np.load('gdrive/My Drive/dataset/dataset6/testy.npy')
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/drive
Mounted at /content/gdrive
(66542, 128, 12) (66542, 2) (7600, 128, 12) (7600, 2)


In [73]:
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.20, random_state=42)

In [74]:
input_shape = x_train.shape[1:]
inp = Input(shape=x_train.shape[1:])
x = SpatialDropout1D(0.2)(inp)
x = TCN(128,dilations = [1, 2, 4], return_sequences=True)(x)
x = TCN(64,dilations = [1, 2, 4], return_sequences=True)(x)
avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
    
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.2)(conc)
outp = Dense(2, activation="softmax")(conc)    
model = Model(inputs=inp, outputs=outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "model_14"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 128, 12)]    0           []                               
                                                                                                  
 spatial_dropout1d_14 (SpatialD  (None, 128, 12)     0           ['input_15[0][0]']               
 ropout1D)                                                                                        
                                                                                                  
 tcn_28 (TCN)                   (None, 128, 128)     252800      ['spatial_dropout1d_14[0][0]']   
                                                                                                  
 tcn_29 (TCN)                   (None, 128, 64)      94656       ['tcn_28[0][0]']          

In [75]:
history = model.fit(x_train, y_train, epochs=50, verbose=True, validation_data=(x_validation, y_validation), batch_size=128)

Epoch 1/50
416/416 [==============================] - 34s 75ms/step - loss: 0.5437 - accuracy: 0.7694 - val_loss: 0.3700 - val_accuracy: 0.8249
Epoch 2/50
416/416 [==============================] - 30s 73ms/step - loss: 0.3835 - accuracy: 0.8239 - val_loss: 0.2702 - val_accuracy: 0.8895
Epoch 3/50
416/416 [==============================] - 30s 73ms/step - loss: 0.3521 - accuracy: 0.8440 - val_loss: 0.2543 - val_accuracy: 0.8974
Epoch 4/50
416/416 [==============================] - 31s 73ms/step - loss: 0.3330 - accuracy: 0.8509 - val_loss: 0.2719 - val_accuracy: 0.8858
Epoch 5/50
416/416 [==============================] - 30s 73ms/step - loss: 0.3229 - accuracy: 0.8566 - val_loss: 0.2500 - val_accuracy: 0.8961
Epoch 6/50
416/416 [==============================] - 30s 73ms/step - loss: 0.3025 - accuracy: 0.8668 - val_loss: 0.2207 - val_accuracy: 0.9110
Epoch 7/50
416/416 [==============================] - 30s 73ms/step - loss: 0.2942 - accuracy: 0.8702 - val_loss: 0.2415 - val_accuracy:

In [76]:
loss1, accuracy1 = model.evaluate(x_validation, y_validation, verbose=False)
print("Validation Accuracy:  {:.4f}".format(accuracy1))
loss1, accuracy1 = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy: {:.4f}".format(accuracy1))

Validation Accuracy:  0.9542
Testing Accuracy: 0.9128
